# Astrobrowser - Spitzer Photometry and Accurate Rotation Curves (SPARC)

Retrieve flux-calibrated photometric images in SDSS $g$, $r$, and $i$ bands (based on Giordano et al. zero points) and derive surface brightness profiles.

# 1. Initialisation

External libraries

In [1]:
#%matplotlib ipympl
import os
import numpy as np
from scipy import ndimage
from matplotlib import pyplot as plt
from matplotlib import colors
from matplotlib.ticker import AutoMinorLocator
from matplotlib.backends.backend_pdf import PdfPages
from astropy.coordinates import SkyCoord, get_icrs_coordinates
from astropy.io import fits
from astropy.wcs import WCS
from astropy.table import QTable
from astropy import units as u
from astropy import constants as c
#from photutils.aperture import EllipticalAperture, EllipticalAnnulus
#from photutils.isophote import EllipseGeometry, Ellipse
from scripts import astrobrowser

In [2]:
%load_ext autoreload
%autoreload 2

Utility functions

In [3]:
show_plots = True
show_plots = False
plt.ioff()

In [4]:
def new_figure(fig_name, figsize=(12, 8), nrows=1, ncols=1, sharex='col', sharey='row', gridspec_kw={'hspace': 0, 'wspace': 0}, suptitle=True):
    plt.close(fig_name)
    fig = plt.figure(fig_name, figsize=figsize, layout="constrained")
    axes = fig.subplots(nrows=nrows, ncols=ncols, squeeze=False,
                        sharex=sharex, sharey=sharey,
                        gridspec_kw=gridspec_kw
                       )
    #fig.set_tight_layout(True)
    for ax in axes.flat:
        ax.xaxis.set_minor_locator(AutoMinorLocator())
        ax.yaxis.set_minor_locator(AutoMinorLocator())
        ax.tick_params(which='both', bottom=True, top=True, left=True, right=True)
        ax.tick_params(which='major', direction='inout', length=8, grid_alpha=.3)
        ax.tick_params(which='minor', direction='in', length=2, grid_alpha=.1)
        ax.grid(True, which='both')

    if suptitle is True:
        fig.suptitle(fig_name)
    elif suptitle is not False and suptitle is not None:
        fig.suptitle(suptitle)
    
    return fig, axes

In [5]:
def draw_ellipse(ax, x0, y0, a, b, pa):
    theta = np.linspace(0, 2*np.pi, 361)
    along_major_axis = a*np.cos(theta)
    along_minor_axis = b*np.sin(theta)
    ra = x0 + along_major_axis * np.cos(pa + np.pi/2) + along_minor_axis *np.cos(pa + np.pi)
    dec = y0 + along_major_axis * np.sin(pa + np.pi/2) + along_minor_axis *np.sin(pa + np.pi)
    ax.plot(ra, dec, 'k--')

In [6]:
def test_dir(dir_name):
    if not os.path.isdir(dir_name):
        print(f'>> WARNING: Creating directory "{dir_name}"')
        os.makedirs(dir_name)
    return(dir_name)

Directories

In [7]:
input_dir = 'SPARC'
output_dir = test_dir(os.path.join(input_dir, 'output'))
maps_dir = test_dir(os.path.join(output_dir, 'maps'))
plots_dir = test_dir(os.path.join(output_dir, 'plots'))
profiles_dir = test_dir(os.path.join(output_dir, 'profiles'))


# 2. Observations

## SPARC catalogue
I had to edit the file a bit to conform to the Machine Readable Table (MRT) format

In [8]:
SPARC_catalogue = QTable.read(os.path.join(input_dir, 'SPARC_clean.mrt'), format='mrt')

In [9]:
SPARC_catalogue

Galaxy,T,D,e_D,f_D,Inc,e_Inc,L[3.6],e_L[3.6],Reff,SBeff,Rdisk,SBdisk,MHI,RHI,Vflat,e_Vflat,Q,Ref.
,,Mpc,Mpc,,deg,deg,1e+09 Lsun,1e+09 Lsun,kpc,Lsun / pc2,kpc,Lsun / pc2,1e+09 Msun,kpc,km / s,km / s,,
str8,int64,float64,float64,int64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,int64,str9
UGC07125,9,19.8,5.9,1,90.0,3.0,2.712,0.08,3.92,28.11,3.38,103.0,4.629,23.04,65.2,2.1,1,"Sw09,Sw02"
UGC07151,6,6.87,0.34,2,90.0,3.0,2.284,0.025,2.17,76.71,1.25,965.67,0.616,6.39,73.5,2.8,1,"Sw09,Sw02"
UGC07232,10,2.83,0.17,2,59.0,5.0,0.113,0.002,0.46,83.34,0.29,227.42,0.046,1.54,0.0,0.0,2,"Sw09,Sw02"
UGC07261,8,13.1,3.93,1,30.0,10.0,1.753,0.048,2.66,39.16,1.2,566.02,1.388,10.1,74.7,3.4,2,"Sw09,Sw02"
UGC07323,8,8.0,2.4,1,47.0,3.0,4.109,0.042,3.26,61.49,2.26,283.68,0.722,7.14,0.0,0.0,1,"Sw09,Sw02"
UGC07399,8,8.43,2.53,1,55.0,3.0,1.156,0.024,1.27,113.98,1.64,135.78,0.745,7.85,103.0,3.3,1,"Sw09,Sw02"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
UGC11914,2,16.9,5.1,1,31.0,5.0,150.028,0.553,3.12,2425.66,2.44,3168.33,0.888,9.26,288.1,10.5,1,"No07,No05"


## SDSS HiPS skymaps

In [10]:
class HiPS_skymap(object):
    
    def __init__(self, hips_service_url, units, beam=None):
        '''Intensity map in Hierarchical Progressive Survey (HiPS) format'''
        
        print(f'> {hips_service_url}')
        self.url = hips_service_url
        self.properties = astrobrowser.get_hips_proprties(hips_service_url)
        if self.properties is None:
            print('  ERROR: HiPS properties not available!')
            raise -1
        if 'hips_pixel_scale' in self.properties:
            self.hips_pixel = float(self.properties['hips_pixel_scale']) * u.deg
        else:
            print('  ERROR: HiPS pixel size not available!')
            raise -1

        if beam is None:
            if 's_pixel_scale' in self.properties:
                original_pixel = float(self.properties['s_pixel_scale']) * u.deg
                self.beam = original_pixel**2
            else:
                self.beam = self.hips_pixel**2
                print(f'  WARNING: original pixel size not available! using HiPS size = {self.hips_pixel.to_value(u.arcsec)} arcsec')
        else:
            self.beam = beam
            original_pixel_beam = np.sqrt(beam)
            if 's_pixel_scale' in self.properties:
                original_pixel_properties = float(self.properties['s_pixel_scale']) * u.deg
                if not u.isclose(original_pixel_beam, original_pixel_properties):
                    print(f'  WARNING: {original_pixel_beam} is different from {original_pixel_properties} ({original_pixel_properties.to(original_pixel_beam.unit)})')

        self.intensity_units = units
        if u.get_physical_type(units) == 'spectral flux density':
            self.intensity_units = units / self.beam
        
        print(f'  HiPS pixel = {self.hips_pixel.to(u.arcsec):.4f}, original = {np.sqrt(self.beam).to(u.arcsec):.4f}',
              f'; units = {self.intensity_units.to(u.uJy/u.arcsec**2):.2f} = {self.intensity_units.to(u.MJy/u.sr):.4f}')


These SDSS measurements correpond to surface brightness in nanomaggies per beam (original pixel)

In [11]:
nanomaggies = 3.631e-6*u.Jy
beam = (0.39564 * u.arcsec)**2
pivot_wavelength = {
    'g': 4702.50 * u.Angstrom,
    'r': 6175.58 * u.Angstrom,
    'i': 7489.98 * u.Angstrom,
}
solar_units_factor = (4*np.pi*u.sr) * c.c  # divide by the pilot wavelength to convert from intensity (e.g. nanomaggies/beam) to luminosity surface brightness (Lsun/pc^2)

In [12]:
SDSS_skymaps = {
    'g': HiPS_skymap('https://alasky.cds.unistra.fr/SDSS/DR9/band-g', nanomaggies, beam),
    'r': HiPS_skymap('https://alasky.cds.unistra.fr/SDSS/DR9/band-r', nanomaggies, beam),
    'i': HiPS_skymap('https://alasky.cds.unistra.fr/SDSS/DR9/band-i', nanomaggies, beam),
}

> https://alasky.cds.unistra.fr/SDSS/DR9/band-g
  HiPS pixel = 0.4025 arcsec, original = 0.3956 arcsec ; units = 23.20 uJy / arcsec2 = 0.9869 MJy / sr
> https://alasky.cds.unistra.fr/SDSS/DR9/band-r
  HiPS pixel = 0.4025 arcsec, original = 0.3956 arcsec ; units = 23.20 uJy / arcsec2 = 0.9869 MJy / sr
> https://alasky.cds.unistra.fr/SDSS/DR9/band-i
  HiPS pixel = 0.4025 arcsec, original = 0.3956 arcsec ; units = 23.20 uJy / arcsec2 = 0.9869 MJy / sr


## Download images

In [13]:
def fectch_images(galaxy, skymaps, overwrite=False, fig_name=None, max_size=300):
    """
    Call the AstroBrowser to download HiPS cutouts, or
    read them from disk if they are present.
    """
    cutout_pixel = np.inf
    for band in skymaps:
        if skymaps[band].hips_pixel < cutout_pixel:
            cutout_pixel = skymaps[band].hips_pixel
            
    theta_disk = np.arcsin(galaxy['Rdisk'] / galaxy['D'])
    if theta_disk > max_size*cutout_pixel:
        cutout_pixel = theta_disk / max_size
    
    header = {}
    data = {}
    for band in skymaps:
        cutout_file = os.path.join(maps_dir, f"{galaxy['Galaxy']}_{band}.fits")
        if overwrite or not os.path.isfile(cutout_file):
            print(f"- Downloading {cutout_file}... (please be patient)")
            try:
                position = get_icrs_coordinates(galaxy['Galaxy'])
            except:
                print(f"  Name {galaxy['Galaxy']} not found!")
                header[band] = None
                data[band] = None
                return header, data
            header[band], data[band] = astrobrowser.get_cutout(
                skymaps[band].url,
                position.ra.deg, position.dec.deg,
                4*theta_disk.to_value(u.arcsec), cutout_pixel.to_value(u.arcsec),
                cutout_file, overwrite=True)
        else:
            print(f'- Reading "{cutout_file}"')
            with fits.open(cutout_file) as hdu:
                header[band] = hdu[0].header
                data[band] = hdu[0].data

    if fig_name is not None and header[next(iter(header))] is not None:
        plt.close(fig_name)
        n_bands = len(data)
        fig = plt.figure(fig_name, figsize=(6*n_bands, 5))
        for idx, band in enumerate(data):
            wcs = WCS(header[band])
            ax = fig.add_subplot(1, n_bands, idx+1)#, projection=wcs)
            ax.set_title(band)
            #img = data[band] * (nanomaggies/beam).to_value(3631*u.Jy/u.arcsec**2)
            #im = ax.imshow(img, origin='lower', cmap='nipy_spectral', norm=colors.Normalize(-1e-10, 1e-9))
            img = -2.5*np.log10(data[band] * (nanomaggies/beam).to_value(3631*u.Jy/u.arcsec**2))
            im = ax.imshow(img, origin='lower', interpolation='nearest', cmap='nipy_spectral', norm=colors.Normalize(17.5, 26.5))
            cb = plt.colorbar(im, ax=ax, shrink=.9)
        fig.savefig(os.path.join(plots_dir, f'{fig_name}.png'), facecolor='white')
        if not show_plots:
            plt.close(fig_name)

    return header, data

# 3. Analysis

## Mass-to-light ratios
Obtained from Garcia-Benito et al.

In [14]:
mass_to_light_ratio = {}
mass_to_light_ratio['ggr'] = (-0.88, 1.88)
mass_to_light_ratio['ggi'] = (-0.99, 1.29)
mass_to_light_ratio['gri'] = (-1.08, 3.74)
mass_to_light_ratio['rgr'] = (-0.70, 1.49)
mass_to_light_ratio['rgi'] = (-0.79, 1.03)
mass_to_light_ratio['rri'] = (-0.86, 2.98)
mass_to_light_ratio['igr'] = (-0.69, 1.31)
mass_to_light_ratio['igi'] = (-0.77, 0.90)
mass_to_light_ratio['iri'] = (-0.83, 2.60)

In [15]:
def estimate_stellar_surface_density(data, fig_name=None, profiles_table=None):
    """
    Estimate stellar surface density from SDSS images,
    according to RGB M/L ratios.
    """
    mass = np.full((len(mass_to_light_ratio),)+data[next(iter(data))].shape, np.nan)
    for idx, mass_map in enumerate(mass_to_light_ratio):
        if mass_map[0] in data and mass_map[1] in data and mass_map[2] in data:
            a, b = mass_to_light_ratio[mass_map]
            mass[idx] = 10**(a - b * 2.5 * np.log10(data[mass_map[1]]/data[mass_map[2]]))
            mass[idx] *= data[mass_map[0]]
            mass[idx] *= (solar_units_factor*nanomaggies/beam/pivot_wavelength[mass_map[0]]).to_value(u.Lsun/u.pc**2)
    
    if profiles_table is not None:
        for idx, mass_map in enumerate(mass_to_light_ratio):
            profiles_table.add_column(mass[idx] << u.Msun/u.pc**2, name=f'{mass_map[0]} and ({mass_map[1]}-{mass_map[2]})')
    
    if fig_name is not None:
        plt.close(fig_name)
        fig = plt.figure(fig_name, figsize=(14, 12))
        #norm = colors.Normalize(1, 5)  # for M/L ratio
        norm = colors.LogNorm(3, 3e3)  # for surface density (Msun/pc^2)
        for idx, mass_map in enumerate(mass_to_light_ratio):
            if mass_map[0] in data and mass_map[1] in data and mass_map[2] in data:
                ax = fig.add_subplot(3, 3, idx+1)
                ax.set_title(f'$\Sigma$ from {mass_map[0]} and ({mass_map[1]}-{mass_map[2]})')
                im = ax.imshow(mass[idx], origin='lower', cmap='nipy_spectral', norm=norm)
                cb = plt.colorbar(im, ax=ax, shrink=.9)
        fig.savefig(os.path.join(plots_dir, f'{fig_name}.png'), facecolor='white')
        if not show_plots:
            plt.close()
    
    return np.median(mass, axis=0), np.std(np.log10(mass), axis=0)

## Fit ellipse

In [16]:
class Ellipse(object):
    
    def __init__(self,
                 data,
                 center_seed=None, recenter=True, inner_radius=None, max_iter=10,
                 fig_name=None):
        
        # Set nan and negatives to 0
        valid_data = data.copy()
        valid_data = np.where(valid_data > 0, valid_data, 0)

        # Define inner region
        if center_seed is None:
            self.y0, self.x0 = data.shape
            self.x0 /= 2
            self.y0 /= 2
        else:
            self.x0, self.y0 = center_seed
        if inner_radius is None:
            inner_radius = np.min(valid_data.shape) // 8
        x = np.arange(valid_data.shape[1]) - self.x0
        y = np.arange(valid_data.shape[0]) - self.y0
        r = np.sqrt((x**2)[np.newaxis, :] + (y**2)[:, np.newaxis])
        self.inner_mask = r < inner_radius

        # Find isophote and recenter, if requested
        iteration = 0
        cm_moved = np.inf
        while cm_moved > 1 and iteration < max_iter:
            iteration += 1
            mask, x_new, y_new = self._find_isophote(valid_data, inner_radius)
            if recenter:
                cm_moved = np.sqrt((x_new-self.x0)**2 + (y_new-self.y0)**2)
                self.x0 = x_new.copy()
                self.y0 = y_new.copy()
                print(f'  Center at ({self.x0:.2f}, {self.y0:.2f}) moved {cm_moved:.4g} pix')
            else:
                cm_moved = 0
        print(f'> Ellipse centered at ({self.x0:.2f}, {self.y0:.2f})')
        x = np.arange(valid_data.shape[1]) - self.x0
        y = np.arange(valid_data.shape[0]) - self.y0
        r = np.sqrt((x**2)[np.newaxis, :] + (y**2)[:, np.newaxis])

        # Find mean isophote radius as a function of polar angle
        theta_r = np.linspace(0, np.pi, 181)
        mean_r = np.empty_like(theta_r)
        dummy_r = np.arange(1, 2*inner_radius)
        for i, zz in enumerate(theta_r):
            x_i = (self.x0 + dummy_r * np.cos(zz)).astype(int)
            y_i = (self.y0 + dummy_r * np.sin(zz)).astype(int)
            total_weight = np.sum(mask[y_i, x_i])
            mean_r_positive = np.sum(mask[y_i, x_i] * r[y_i, x_i]) / total_weight
            x_i = (self.x0 - dummy_r * np.cos(zz)).astype(int)
            y_i = (self.y0 - dummy_r * np.sin(zz)).astype(int)
            total_weight = np.sum(mask[y_i, x_i])
            mean_r_negative = np.sum(mask[y_i, x_i] * r[y_i, x_i]) / total_weight
            # Geometric mean of both sides
            mean_r[i] = np.sqrt(mean_r_positive * mean_r_negative)

        # Fit ellipse
        inv_r2 = 1/mean_r**2
        mean_value = np.mean(inv_r2)
        coeff_cos = np.mean(inv_r2 * np.cos(2*theta_r)) / np.mean(np.cos(2*theta_r)**2)
        coeff_sin = np.mean(inv_r2 * np.sin(2*theta_r)) / np.mean(np.sin(2*theta_r)**2)
        #model = mean_value + coeff_cos*np.cos(2*theta_r) + coeff_sin*np.sin(2*theta_r)
        amplitude = - np.sqrt(coeff_cos**2 + coeff_sin**2)
        coeff_cos /= amplitude
        coeff_sin /= amplitude
        #model = mean_value + amplitude * (coeff_cos*np.cos(2*theta_r) + coeff_sin*np.sin(2*theta_r))
        if coeff_sin < 0:
            self.theta_0 = np.pi - np.arccos(coeff_cos) / 2
        else:
            self.theta_0 = np.arccos(coeff_cos) / 2
        model = mean_value + amplitude * np.cos(2*(theta_r - self.theta_0))
        self.a = 1 / np.sqrt(mean_value + amplitude)
        self.b = 1 / np.sqrt(mean_value - amplitude)
        self.e = 1 - self.b/self.a
        print(f'  (a, b, theta_0) = ({self.a:.2f} pix, {self.b:.2f} pix, {self.theta_0*180/np.pi:.2f} deg)')
        print('> WARNING: pixel units')
        print('  TODO: convert to inclination and PA')
        
        # Deprojection / profiles:
        r[r <= 0.] = 1e-6
        theta = np.where(y[:, np.newaxis] >= 0, np.arccos(x[np.newaxis, :]/r), 2*np.pi - np.arccos(x[np.newaxis, :]/r))
        theta -= self.theta_0
        theta[theta < 0] += 2*np.pi
        self.r_0 = r * np.sqrt(np.cos(theta)**2 + (np.sin(theta) * self.a/self.b)**2)

        # Plot figure
        if fig_name is not None:
            fig, axes = new_figure(fig_name, nrows=2, ncols=2, sharey=False,)
            
            ax = axes[0, 0]
            im = ax.imshow(mask, origin='lower', interpolation='nearest')
            x_r = self.x0 + mean_r * np.cos(theta_r)
            y_r = self.y0 + mean_r * np.sin(theta_r)
            ax.plot(x_r, y_r, 'k-')
            x_r = self.x0 + mean_r * np.cos(theta_r+np.pi)
            y_r = self.y0 + mean_r * np.sin(theta_r+np.pi)
            ax.plot(x_r, y_r, 'k-')
            ax.plot(self.x0, self.y0, 'ko')
            ax.contour(self.inner_mask, colors='w', linestyles=':')
            cb = plt.colorbar(im, ax=ax, shrink=.75)
            
            ax = axes[0, 1]
            ax.set_ylabel(r'1 / radius$^2$ [pix$^{-2}$]')
            ax.plot(theta_r * 180/np.pi, inv_r2, 'k-', alpha=.2, label=f'isophote {self.isophote_mean:.4g} $\pm$ {self.isophote_mad:.4g}')
            ax.plot(theta_r * 180/np.pi, model, 'k--',
                    label=f'(a, b, theta_0) = ({self.a:.2f} pix, {self.b:.2f} pix, {self.theta_0*180/np.pi:.2f} deg)')
            ax.axvline(self.theta_0 * 180/np.pi, c='k', ls=':')
            ax.legend()
            
            ax = axes[1, 0]
            im = ax.imshow(valid_data, origin='lower', interpolation='nearest', vmax=self.isophote_mean+2*self.isophote_mad, cmap='terrain')
            self.plot(ax, self.a)
            ax.plot(self.x0, self.y0, 'ko')
            ax.contour(self.inner_mask, colors='w', linestyles=':')
            cb = plt.colorbar(im, ax=ax, shrink=.75)
            
            ax = axes[1, 1]
            ax.set_ylabel('radius [pix]')
            ax.set_xlabel('theta [deg]')
            ax.plot(theta_r * 180/np.pi, mean_r, 'k-', alpha=.2, label=f'isophote {self.isophote_mean:.4g} $\pm$ {self.isophote_mad:.4g}')
            ax.plot(theta_r * 180/np.pi, 1/np.sqrt(model), 'k--',
                    label=f'(a, b, theta_0) = ({self.a:.2f} pix, {self.b:.2f} pix, {self.theta_0*180/np.pi:.2f} deg)')
            ax.axvline(self.theta_0 * 180/np.pi, c='k', ls=':')
            ax.legend()
            
            
            fig.savefig(os.path.join(plots_dir, f'{fig_name}.png'), facecolor='white')
            if not show_plots:
                plt.close()

                
    def _find_isophote(self, valid_data, inner_radius):
        # Maps of polar coordinates (r and theta)
        x = np.arange(valid_data.shape[1]) - self.x0
        y = np.arange(valid_data.shape[0]) - self.y0
        r = np.sqrt((x**2)[np.newaxis, :] + (y**2)[:, np.newaxis])
        theta = np.arccos(x/r)
        theta[y < 0] = np.pi - theta[y < 0]

        # Mean intensity and median absolute deviation
        self.isophote_mean = np.nanmean(valid_data[r < inner_radius])
        self.isophote_mad = np.nanmedian(np.fabs(valid_data[r < inner_radius] - self.isophote_mean))
        
        # Define isophote
        mask = np.fabs(valid_data - self.isophote_mean) < self.isophote_mad
        mask = ndimage.median_filter(mask, 3)
        print(f'  Isophote: {self.isophote_mean:.4g} +- {self.isophote_mad:.4g} => {np.count_nonzero(mask)} pix')
        
        # Recenter
        x_cm = self.x0 + np.sum(x[np.newaxis, :] * mask * self.inner_mask) / np.sum(mask * self.inner_mask)
        y_cm = self.y0 + np.sum(y[:, np.newaxis] * mask * self.inner_mask) / np.sum(mask * self.inner_mask)

        return mask, x_cm, y_cm


    def plot(self, ax, radius, style='k--'):
        theta = np.linspace(0, 2*np.pi, 361)
        for a in np.atleast_1d(radius):
            along_major_axis = a * np.cos(theta)
            along_minor_axis = (a*self.b/self.a) * np.sin(theta)
            ra = self.x0 + along_major_axis * np.cos(self.theta_0) + along_minor_axis * np.cos(self.theta_0 + np.pi/2)
            dec = self.y0 + along_major_axis * np.sin(self.theta_0) + along_minor_axis * np.sin(self.theta_0 + np.pi/2)
            ax.plot(ra, dec, style)
    
    
    def get_profile(self, data, fig_name=None):
        r_0_bins = np.arange(1 + np.sqrt(np.min(data.shape)/2))**2
        r_0_mid = (r_0_bins[:-1] + r_0_bins[1:]) / 2
        median_profile = np.empty(r_0_bins.size-1)
        upper_profile = np.empty_like(median_profile)
        lower_profile = np.empty_like(median_profile)
        for i in range(r_0_bins.size-1):
            r_inner = r_0_bins[i]
            r_outer = r_0_bins[i+1]
            try:
                lower_profile[i], median_profile[i], upper_profile[i] = np.nanpercentile(
                    data[(self.r_0 >= r_inner) & (self.r_0 <= r_outer)], [16, 50, 84])
            except:
                lower_profile[i], median_profile[i], upper_profile[i] = (np.nan, np.nan, np.nan)

        if fig_name is not None:
            fig, axes = new_figure(f'{fig_name}_profile', figsize=(16, 4), ncols=4, sharey=False, sharex=False,
                                   gridspec_kw={'width_ratios': [1, 1, 1, 2]})
            ax = axes[0, 0]
            ax.set_title('data')
            im = ax.imshow(data, origin='lower', interpolation='nearest', cmap='nipy_spectral', norm=colors.LogNorm())
            cb = plt.colorbar(im, ax=ax, shrink=.5)
            self.plot(ax, self.a)
            ax = axes[0, 1]
            ax.set_title('model')
            model = np.interp(self.r_0, r_0_mid, median_profile)
            im = ax.imshow(model, origin='lower', interpolation='nearest', cmap='nipy_spectral', norm=im.norm)
            cb = plt.colorbar(im, ax=ax, shrink=.5)
            self.plot(ax, self.a)
            ax = axes[0, 2]
            ax.set_title('residual')
            residual = data-model
            mad = np.nanmedian(np.fabs(residual))
            im = ax.imshow(residual, origin='lower', interpolation='nearest', cmap='Spectral', norm=colors.Normalize(-5*mad, 5*mad))
            cb = plt.colorbar(im, ax=ax, shrink=.5)
            self.plot(ax, self.a)
            
            ax = axes[0, 3]
            ax.set_title('radial profile')
            ax.plot(self.r_0.ravel(), data.ravel(), 'c.', alpha=.05)
            ax.plot(r_0_mid, median_profile, 'r-+')
            ax.fill_between(r_0_mid, lower_profile, upper_profile, color='k', alpha=.5)
            #ax.set_ylim(-.1, .1)
            ax.set_yscale('log')
            fig.savefig(os.path.join(plots_dir, f"{fig_name}_profile.png"), facecolor='white')     
            if not show_plots:
                plt.close()
        
        return r_0_bins, median_profile, lower_profile, upper_profile

#elllipse = Ellipse(surface_density_map, fig_name='kk')

## Main loop

In [17]:
for galaxy in SPARC_catalogue:
    if np.ma.is_masked(galaxy['Galaxy']):
        continue

    # Cutouts
    header, data = fectch_images(galaxy, SDSS_skymaps, fig_name=f"{galaxy['Galaxy']}_cutouts")
    if header[next(iter(header))] is None:
        continue

    # Maps
    surface_density_map, surface_density_err = estimate_stellar_surface_density(data, fig_name=f"{galaxy['Galaxy']}_mass-to-light")
    hdr = header[next(iter(header))]
    hdr['BUNIT'] = 'M_sun / pc^2'
    hdr['COMMENT'] = 'M/L based on Garcia-Benito et al. (2019)'
    wcs = WCS(hdr)
    fits.PrimaryHDU(header=hdr, data=surface_density_map).writeto(
        os.path.join(maps_dir, f"{galaxy['Galaxy']}_surface_density.fits"), overwrite=True, output_verify='fix')

    # Profiles
    ellipse = Ellipse(surface_density_map, fig_name=f"{galaxy['Galaxy']}_ellipse")
    #if np.isnan(ellipse.theta_0):
    #    continue
    r_bins = {}
    median_profile = {}
    for band in data:
        r_bins[band], median_profile[band] = ellipse.get_profile(data[band], fig_name=f"{galaxy['Galaxy']}_{band}")[:2]
    r_mid = r_bins['g']
    r_mid = (r_mid[1:] + r_mid[:-1]) / 2
    
    ra_mid = ellipse.x0 + r_mid * np.cos(ellipse.theta_0)
    dec_mid = ellipse.y0 + r_mid * np.sin(ellipse.theta_0)
    coord_mid = wcs.pixel_to_world(ra_mid, dec_mid)
    coord_centre = wcs.pixel_to_world(ellipse.x0, ellipse.y0)
    theta_mid = coord_centre.separation(coord_mid).to(u.arcsec)

    profiles_table = QTable()
    profiles_table.add_column(theta_mid, name='theta')
    surface_density_profile, surface_density_profile_err = estimate_stellar_surface_density(median_profile, None, profiles_table)
    profiles_table.add_column(surface_density_profile << u.Msun/u.pc**2, name='median')
    profiles_table.add_column(surface_density_profile_err << u.dex, name='std')    
    profiles_table.write(os.path.join(profiles_dir, f"{galaxy['Galaxy']}_surface_density.csv"), overwrite=True)
    
    # Plot median surface density and uncertainty

    fig_name = f"{galaxy['Galaxy']}_surface_density"
    fig = plt.figure(fig_name, figsize=(20, 5))
    theta_disk = np.arcsin(galaxy['Rdisk'] / galaxy['D'])
    '''
    theta_disk = np.min(surface_density_map.shape) / 4
    aper = EllipticalAperture((ellipse.isophote.x0, ellipse.isophote.y0),
                              ellipse.isophote.sma,
                              ellipse.isophote.sma * (1 - ellipse.isophote.eps),
                              ellipse.isophote.pa)
    aper2 = EllipticalAperture((ellipse.isophote.x0, ellipse.isophote.y0),
                               theta_disk, theta_disk * (1 - ellipse.isophote.eps), ellipse.isophote.pa)
    '''

    ax = fig.add_subplot(141, projection=wcs)
    ax.set_title('$\Sigma_\star$ [M$_\odot$/pc$^2$]')
    im = ax.imshow(surface_density_map, origin='lower', cmap='nipy_spectral', norm=colors.LogNorm(3, 3e3))
    #aper.plot(ax, color='k')
    #aper2.plot(ax, color='k', ls='--', lw=2)
    ellipse.plot(ax, ellipse.a)
    cb = plt.colorbar(im, ax=ax, shrink=.5)
    
    ax = fig.add_subplot(142, projection=wcs)
    ax.set_title('uncertainty $\Delta\log_{10}\Sigma_\star$ [dex]')
    im = ax.imshow(surface_density_err, cmap='Spectral_r', vmin=.05, vmax=.35)
    #aper.plot(ax, color='k')
    #aper2.plot(ax, color='k', ls='--', lw=2)
    ellipse.plot(ax, ellipse.a)
    cb = plt.colorbar(im, ax=ax, shrink=.5)
    
    ax = fig.add_subplot(143, position=[.55, .23, .4, .55])
    ax.set_ylabel(r'$\Sigma$ [M$_\odot$ / pc$^2$]')
    ax.set_xlabel(r'$R$ [arcsec]')
    ax.set_yscale('log')
    ra_pix = ellipse.x0 + ellipse.r_0.ravel() * np.cos(ellipse.theta_0)
    dec_pix = ellipse.y0 + ellipse.r_0.ravel() * np.sin(ellipse.theta_0)
    coord_pix = wcs.pixel_to_world(ra_pix, dec_pix)
    theta_pix = coord_centre.separation(coord_pix)
    #pixscale = wcs.proj_plane_pixel_scales()[0].to_value(u.arcsec)
    #ax.plot(ellipse.r_0.ravel() * pixscale, surface_density_map.ravel(), 'k.', alpha=.05)
    ax.plot(theta_pix.to_value(u.arcsec), surface_density_map.ravel(), 'c.', alpha=.05)
    ax.plot(theta_mid.to_value(u.arcsec), surface_density_profile, 'r-+')
    factor = 10**surface_density_profile_err
    ax.fill_between(theta_mid.to_value(u.arcsec), surface_density_profile/factor, surface_density_profile*factor, color='k', alpha=.5)
    ax.axvline(theta_disk.to_value(u.arcsec), c='k', ls=':')
    ax.set_ylim(.3, 3e4)
    ax.set_xlim(-.1*theta_disk.to_value(u.arcsec), 3*theta_disk.to_value(u.arcsec))
    
    fig.savefig(os.path.join(plots_dir, f"{fig_name}.png"), facecolor='white')
    if not show_plots:
        plt.close()


- Reading "SPARC/output/maps/UGC07125_g.fits"
- Downloading SPARC/output/maps/UGC07125_r.fits... (please be patient)
http://astrobrowser.ft.uam.es/api/cutout?hipsbaseuri=https://alasky.cds.unistra.fr/SDSS/DR9/band-r&radeg=182.172975&decdeg=36.8022972&radiusasec=140.84&pxsizeasec=0.40


/tmp/ipykernel_301897/1617722128.py:49: RuntimeWarning: invalid value encountered in log10
  img = -2.5*np.log10(data[band] * (nanomaggies/beam).to_value(3631*u.Jy/u.arcsec**2))


  Saving "SPARC/output/maps/UGC07125_r.fits"
- Reading "SPARC/output/maps/UGC07125_i.fits"


/tmp/ipykernel_301897/322829930.py:10: RuntimeWarning: invalid value encountered in log10
  mass[idx] = 10**(a - b * 2.5 * np.log10(data[mass_map[1]]/data[mass_map[2]]))
/tmp/ipykernel_301897/322829930.py:33: RuntimeWarning: invalid value encountered in log10
  return np.median(mass, axis=0), np.std(np.log10(mass), axis=0)


  Isophote: 60.51 +- 51.63 => 27873 pix
  Center at (347.96, 359.93) moved 8.709 pix
  Isophote: 60.49 +- 51.52 => 27543 pix
  Center at (347.89, 359.98) moved 0.08162 pix
> Ellipse centered at (347.89, 359.98)
  (a, b, theta_0) = (77.20 pix, 23.42 pix, 172.81 deg)
> WARNING: pixel units
  TODO: convert to inclination and PA
- Downloading SPARC/output/maps/UGC07151_g.fits... (please be patient)
http://astrobrowser.ft.uam.es/api/cutout?hipsbaseuri=https://alasky.cds.unistra.fr/SDSS/DR9/band-g&radeg=182.4941708&decdeg=46.4571806&radiusasec=150.12&pxsizeasec=0.40


...

/tmp/ipykernel_301897/1617722128.py:49: RuntimeWarning: invalid value encountered in log10
  img = -2.5*np.log10(data[band] * (nanomaggies/beam).to_value(3631*u.Jy/u.arcsec**2))
/tmp/ipykernel_301897/322829930.py:10: RuntimeWarning: invalid value encountered in log10
  mass[idx] = 10**(a - b * 2.5 * np.log10(data[mass_map[1]]/data[mass_map[2]]))
/tmp/ipykernel_301897/322829930.py:33: RuntimeWarning: invalid value encountered in log10
  return np.median(mass, axis=0), np.std(np.log10(mass), axis=0)


  Isophote: 66.21 +- 47.81 => 16471 pix
  Center at (309.58, 304.98) moved 4.516 pix
  Isophote: 75.42 +- 56.46 => 16782 pix
  Center at (309.23, 305.08) moved 0.3687 pix
> Ellipse centered at (309.23, 305.08)
  (a, b, theta_0) = (43.10 pix, 29.34 pix, 135.06 deg)
> WARNING: pixel units
  TODO: convert to inclination and PA
- Downloading SPARC/output/maps/UGCA281_g.fits... (please be patient)
http://astrobrowser.ft.uam.es/api/cutout?hipsbaseuri=https://alasky.cds.unistra.fr/SDSS/DR9/band-g&radeg=186.56539&decdeg=48.49398&radiusasec=249.84&pxsizeasec=0.40


  Saving "SPARC/output/maps/UGCA281_g.fits"
- Downloading SPARC/output/maps/UGCA281_r.fits... (please be patient)
http://astrobrowser.ft.uam.es/api/cutout?hipsbaseuri=https://alasky.cds.unistra.fr/SDSS/DR9/band-r&radeg=186.56539&decdeg=48.49398&radiusasec=249.84&pxsizeasec=0.40
  Saving "SPARC/output/maps/UGCA281_r.fits"
- Downloading SPARC/output/maps/UGCA281_i.fits... (please be patient)
http://astrobrowser.ft.uam.es/api/cutout?hipsbaseuri=https://alasky.cds.unistra.fr/SDSS/DR9/band-i&radeg=186.56539&decdeg=48.49398&radiusasec=249.84&pxsizeasec=0.40
  Saving "SPARC/output/maps/UGCA281_i.fits"


/tmp/ipykernel_301897/1617722128.py:49: RuntimeWarning: invalid value encountered in log10
  img = -2.5*np.log10(data[band] * (nanomaggies/beam).to_value(3631*u.Jy/u.arcsec**2))
/tmp/ipykernel_301897/322829930.py:10: RuntimeWarning: invalid value encountered in log10
  mass[idx] = 10**(a - b * 2.5 * np.log10(data[mass_map[1]]/data[mass_map[2]]))
/tmp/ipykernel_301897/322829930.py:33: RuntimeWarning: invalid value encountered in log10
  return np.median(mass, axis=0), np.std(np.log10(mass), axis=0)
/tmp/ipykernel_301897/4000349774.py:140: RuntimeWarning: invalid value encountered in divide
  theta = np.arccos(x/r)


  Isophote: 49.95 +- 49.44 => 126089 pix
  Center at (622.63, 622.28) moved 3.606 pix
  Isophote: 49.5 +- 48.99 => 126071 pix
  Center at (622.63, 622.28) moved 0 pix
> Ellipse centered at (622.63, 622.28)
  (a, b, theta_0) = (119.79 pix, 48.49 pix, 171.06 deg)
> WARNING: pixel units
  TODO: convert to inclination and PA
- Downloading SPARC/output/maps/UGCA442_g.fits... (please be patient)
http://astrobrowser.ft.uam.es/api/cutout?hipsbaseuri=https://alasky.cds.unistra.fr/SDSS/DR9/band-g&radeg=355.9397875&decdeg=-31.9567694&radiusasec=223.81&pxsizeasec=0.40
ERROR: could not download cutout (most likely, timeout) :^(
- Downloading SPARC/output/maps/UGCA442_r.fits... (please be patient)
http://astrobrowser.ft.uam.es/api/cutout?hipsbaseuri=https://alasky.cds.unistra.fr/SDSS/DR9/band-r&radeg=355.9397875&decdeg=-31.9567694&radiusasec=223.81&pxsizeasec=0.40
ERROR: could not download cutout (most likely, timeout) :^(
- Downloading SPARC/output/maps/UGCA442_i.fits... (please be patient)
http://

In [22]:
profiles_table

theta,g and (g-r),g and (g-i),g and (r-i),r and (g-r),r and (g-i),r and (r-i),i and (g-r),i and (g-i),i and (r-i),median,std
arcsec,solMass / pc2,solMass / pc2,solMass / pc2,solMass / pc2,solMass / pc2,solMass / pc2,solMass / pc2,solMass / pc2,solMass / pc2,solMass / pc2,dex
float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64
0.20000000006610544,44.6671334109143,6.784845875890111,0.27338174108629176,51.19593492272672,11.243741541147058,0.8774147472796263,26.00463071420302,6.924577579059447,0.7520031148777908,6.924577579059447,0.7757973852621529
1.000000000087629,33.70350967120573,4.518360289591443,0.14623061157921718,38.58860409266156,7.661394997416895,0.502347280178696,18.850255452447428,4.599792276838024,0.4292929958832526,4.599792276838024,0.8279654499935379
2.600000000052029,13.512856901793619,1.9064204198911416,0.07175141784659349,15.422220249339508,3.1780410076183205,0.23522507365474274,7.646636165003055,1.9325492095943513,0.20059071799807826,1.9325492095943513,0.7968052318708602
5.000000000024732,7.919855674258005,0.7572842098029109,0.013451529495362068,9.045395999042299,1.3674217807284534,0.05566432873109324,3.976136158098725,0.7661501591959351,0.04721383057056722,0.7661501591959351,0.9702555234576102
8.200000000051903,5.169968121628743,1.1972159627162118,0.10942157075819986,5.919289981240646,1.8182394121531944,0.27239936579703555,3.425571241990682,1.2239588463348232,0.23476542125152958,1.2239588463348232,0.5870494454992564
12.199999999978658,4.495694581057858,1.4633383229558283,0.24211086674742233,5.1634613836318,2.088786386032484,0.5013528280248251,3.32395191627173,1.506852624355713,0.4356223684924446,1.506852624355713,0.4456197825968917
...,...,...,...,...,...,...,...,...,...,...,...
136.99999978497203,7.786730063057963,3.1076784790039174,0.48601843220735463,9.180466826947075,4.498896369849827,1.0237213232340185,6.344225958172324,3.3297536769116345,0.913939435400602,3.3297536769116345,0.427541215025786


In [19]:
SkyCoord(0*u.deg, 60*u.deg).separation(SkyCoord(10*u.arcsec, 60*u.deg)).to_value(u.arcsec), 10*np.cos(60*u.deg)

(4.999999999632745, <Quantity 5.>)

In [20]:
wcs.array_index_to_world([0, 1, 2, 990, 495], [0, 0, 0, 0, 495])#.dec.to(u.arcsec)

<SkyCoord (ICRS): (ra, dec) in deg
    [(186.6699143 , 48.42464377), (186.66991453, 48.42475488),
     (186.66991476, 48.42486599), (186.67014116, 48.53464367),
     (186.58705708, 48.47968909)]>

In [21]:
wcs.pixel_to_world(ra_bins, dec_bins).dec.to_value(u.arcsec)

NameError: name 'ra_bins' is not defined

In [ ]:
theta_mid

In [ ]:
wcs.array_index_to_world(indices[1], indices[0])

# -- Old Ellipse classes

In [ ]:
class old_Ellipse(object):
    
    def __init__(self, data, fig_name=None):
        self.y0, self.x0 = data.shape
        self.x0 /= 2
        self.y0 /= 2
        n_bins = np.min(data.shape) - 1
        theta = np.linspace(0, 2*np.pi, 361)
        inv_r2 = np.geomspace((2 / n_bins)**2, 1, n_bins)
        r = np.sqrt(1 / inv_r2)
        y = self.y0 + r[:, np.newaxis] * np.cos(theta[np.newaxis, :]) # position angle refers to North
        x = self.x0 - r[:, np.newaxis] * np.sin(theta[np.newaxis, :]) # couter-clockwise
        surface_density_polar = data[y.astype(int), x.astype(int)]
        surface_density_polar = np.where(surface_density_polar > 0, surface_density_polar, 0)

        half_galaxy_radius_pix = n_bins // 8
        inner_mean = np.nanmean(surface_density_polar[r < half_galaxy_radius_pix, :])
        inner_median = np.nanmedian(surface_density_polar[r < half_galaxy_radius_pix, :])
        #isophote = (surface_density_polar > inner_median) & (surface_density_polar < inner_mean)
        #print(f'Isophote between {inner_median:.4g} and {inner_mean:.4g}')
        #mu = (inner_median + inner_mean) / 2
        #sigma = (inner_mean - inner_median) / 2
        #isophote = np.exp(-.5*(surface_density_polar - mu)**2 / sigma**2)
        isophote = np.fabs(surface_density_polar - inner_mean)
        mad = np.nanmedian(isophote)
        print(mad)
        isophote /= mad
        isophote = np.exp(-isophote**2)
        #isophote = (r[:, np.newaxis] < half_galaxy_radius_pix) & (surface_density_polar > inner_mean)
        #isophote[half_galaxy_radius_pix:] = False
        isophote_radius = np.nanmean(np.where(isophote, r[:, np.newaxis], np.nan), axis=0)
        mean_isophote_radius = np.nanmean(isophote_radius)
        a = np.nansum(((isophote_radius-mean_isophote_radius)*np.sin(2*theta))[:-1]) / 180
        b = np.nansum(((isophote_radius-mean_isophote_radius)*np.cos(2*theta))[:-1]) / 180
        amplitude = np.sqrt(a**2 + b**2)
        if a < 0:
            self.position_angle = np.pi - np.arccos(b/amplitude) / 2
        else:
            self.position_angle = np.arccos(b/amplitude) / 2
        print(f' A: {a/amplitude:.2f}, {-np.arcsin(a/amplitude)*180/np.pi:.2f}')
        print(f' B: {b/amplitude:.2f}, {np.arccos(b/amplitude)*180/np.pi:.2f}')
        print(f' theta, A, B: {self.position_angle*180/np.pi:.2f}, {-np.sin(2*self.position_angle):.2f}, {np.cos(2*self.position_angle):.2f} {fig_name}')
        self.axis_ratio = (mean_isophote_radius + amplitude) / (mean_isophote_radius - amplitude)

        if fig_name is not None:
            fig, axes = new_figure(fig_name, nrows=1)
            ax = axes[0, 0]
            ax.set_title('$\Sigma_\star$ [M$_\odot$/pc$^2$]')
            ax.set_ylabel('radius [pix]')
            #im = ax.imshow(surface_density_polar, origin='lower', interpolation='nearest', cmap='nipy_spectral', norm=colors.LogNorm(3, 3e3))
            im = ax.imshow(isophote, origin='lower', interpolation='nearest')
            #ax.contour(surface_density_polar, levels=[inner_median, inner_mean], colors=['k', 'k'])
            #ax.contour(isophote)
            ax.plot(isophote_radius, 'w--')
            #ax.plot(mean_isophote_radius + a*np.sin(2*theta) + b*np.cos(2*theta), 'k--')
            ax.plot(mean_isophote_radius + amplitude*np.cos(2*(theta-self.position_angle)), 'k--', label=f'(a/b, PA) = ({self.axis_ratio:.2f}, {self.position_angle*180/np.pi:.1f})')
            a = half_galaxy_radius_pix
            b = a/self.axis_ratio
            amplitude = (self.axis_ratio - 1) / (self.axis_ratio + 1)
            ax.plot((a+b)/2 * (1 + amplitude*np.cos(2*(theta-self.position_angle))), 'w--')
            a = 2*half_galaxy_radius_pix
            b = a/self.axis_ratio
            amplitude = (self.axis_ratio - 1) / (self.axis_ratio + 1)
            ax.plot((a+b)/2 * (1 + amplitude*np.cos(2*(theta-self.position_angle))), 'w--')
            ax.axhline(half_galaxy_radius_pix, c='k', ls=':')
            ax.axvline(self.position_angle*180/np.pi, c='k', ls=':')
            ax.axvline(self.position_angle*180/np.pi +180, c='k', ls=':')
            cb = plt.colorbar(im, ax=ax, shrink=.75)
            ax.legend()
            ax.set_xlabel('theta [deg]')
            fig.savefig(os.path.join(plots_dir, f'{fig_name}.png'), facecolor='white')
            if not show_plots:
                plt.close()

        # Deprojection / profiles:
        x = np.arange(data.shape[1]) - self.x0
        y = np.arange(data.shape[0]) - self.y0
        r = np.sqrt((x**2)[np.newaxis, :] + (y**2)[:, np.newaxis])
        r[r < 0.1] = .1
        theta = np.where(x < 0, np.arccos(y[:, np.newaxis]/r), 2*np.pi - np.arccos(y[:, np.newaxis]/r))
        theta -= self.position_angle
        theta[theta < 0] += 2*np.pi
        self.theta_0 = np.arctan(np.tan(theta) * self.axis_ratio)
        self.theta_0[(theta >= np.pi/2) & (theta < 1.5*np.pi)] += np.pi
        self.theta_0[self.theta_0 < 0] += 2*np.pi
        self.r_0 = r / np.sqrt(np.cos(self.theta_0)**2 + (np.sin(self.theta_0) / self.axis_ratio)**2)
        
        '''
        fig, axes = new_figure('kk', ncols=2)
        ax = axes[0, 0]
        im = ax.imshow(r_0, origin='lower', interpolation='nearest', cmap='nipy_spectral')
        plt.colorbar(im, shrink=.75)
        ax = axes[0, 1]
        im = ax.imshow(theta_0, origin='lower', interpolation='nearest', cmap='hsv')
        plt.colorbar(im, shrink=.75)
        '''

        
    def plot(self, ax, radius, style='k--'):
        theta = np.linspace(0, 2*np.pi, 361)
        for a in np.atleast_1d(radius):
            along_major_axis = a * np.cos(theta)
            along_minor_axis = (a/self.axis_ratio) * np.sin(theta)
            ra = self.x0 + along_major_axis * np.cos(self.position_angle + np.pi/2) + along_minor_axis *np.cos(self.position_angle + np.pi)
            dec = self.y0 + along_major_axis * np.sin(self.position_angle + np.pi/2) + along_minor_axis *np.sin(self.position_angle + np.pi)
            ax.plot(ra, dec, style)
    
    def get_profile(self, data, fig_name=None):
        r_0_bins = np.arange(1 + np.sqrt(np.min(data.shape)/2))**2
        r_0_mid = (r_0_bins[:-1] + r_0_bins[1:]) / 2
        median_profile = np.empty(r_0_bins.size-1)
        upper_profile = np.empty_like(median_profile)
        lower_profile = np.empty_like(median_profile)
        for i in range(r_0_bins.size-1):
            r_inner = r_0_bins[i]
            r_outer = r_0_bins[i+1]
            try:
                lower_profile[i], median_profile[i], upper_profile[i] = np.nanpercentile(
                    data[(self.r_0 >= r_inner) & (self.r_0 <= r_outer)], [16, 50, 84])
            except:
                lower_profile[i], median_profile[i], upper_profile[i] = (np.nan, np.nan, np.nan)

        if fig_name is not None:
            fig, axes = new_figure(f'{fig_name}_profile', figsize=(16, 4), ncols=4, sharey=False, sharex=False,
                                   gridspec_kw={'width_ratios': [1, 1, 1, 2]})
            ax = axes[0, 0]
            ax.set_title('data')
            im = ax.imshow(data, origin='lower', interpolation='nearest', cmap='nipy_spectral', y=r, norm=colors.LogNorm())
            cb = plt.colorbar(im, ax=ax, shrink=.5)
            ax = axes[0, 1]
            ax.set_title('model')
            model = np.interp(self.r_0, r_0_mid, median_profile)
            im = ax.imshow(model, origin='lower', interpolation='nearest', cmap='nipy_spectral', norm=colors.LogNorm())
            cb = plt.colorbar(im, ax=ax, shrink=.5)
            ax = axes[0, 2]
            ax.set_title('residual')
            residual = data-model
            mad = np.nanmedian(np.fabs(residual))
            im = ax.imshow(residual, origin='lower', interpolation='nearest', cmap='Spectral', norm=colors.Normalize(-5*mad, 5*mad))
            cb = plt.colorbar(im, ax=ax, shrink=.5)
            
            ax = axes[0, 3]
            ax.set_title('radial profile')
            ax.plot(self.r_0.ravel(), data.ravel(), 'c.', alpha=.05)
            ax.plot(r_0_mid, median_profile, 'r-+')
            ax.fill_between(r_0_mid, lower_profile, upper_profile, color='k', alpha=.5)
            #ax.set_ylim(-.1, .1)
            ax.set_yscale('log')
            fig.savefig(os.path.join(plots_dir, f"{fig_name}_profile.png"), facecolor='white')     
            if not show_plots:
                plt.close()
        
        return r_0_bins, median_profile, lower_profile, upper_profile

#old_Ellipse(surface_density_map, 'kk')

In [ ]:
class Isophotes(object):
    
    def __init__(self, data, fig_name=None):
        dx = data.shape[1] // 4
        dy = data.shape[0] // 4
        valid_data = np.where(data > 0, data, 0)
        rms_data = np.sqrt(np.nanmean(valid_data[dy:3*dy, dx:3*dx]**2))
        area = np.count_nonzero(valid_data[dy:3*dy, dx:3*dx] >= rms_data)
        print(area, np.sqrt(area))
        geometry = EllipseGeometry(x0=data.shape[1]/2, y0=data.shape[0]/2,
                                   sma=np.sqrt(area), eps=1e-6, pa=0)
        ellipse = Ellipse(valid_data, geometry, threshold=0)
        self.isophote = ellipse.fit_isophote(sma=geometry.sma, nclip=3)
        
        x = np.arange(data.shape[1]) - self.isophote.x0
        y = np.arange(data.shape[0]) - self.isophote.y0
        r = np.sqrt((x**2)[np.newaxis, :] + (y**2)[:, np.newaxis])
        r[r <= 0.] = 1e-6
        theta = np.where(y[:, np.newaxis] >= 0, np.arccos(x[np.newaxis, :]/r), 2*np.pi - np.arccos(x[np.newaxis, :]/r))
        theta -= self.isophote.pa
        theta[theta < 0] += 2*np.pi
        self.r_0 = r * np.sqrt(np.cos(theta)**2 + (np.sin(theta) / (1 - self.isophote.eps))**2)

    
    def get_profile(self, data, fig_name=None):
        r_0_bins = np.arange(1, 1 + np.sqrt(np.min(data.shape)/2))**2
        r_0_mid = (r_0_bins[:-1] + r_0_bins[1:]) / 2
        median_profile = np.empty(r_0_bins.size-1)
        upper_profile = np.empty_like(median_profile)
        lower_profile = np.empty_like(median_profile)
        #self.r_0 = np.zeros_like(data)
        for i in range(r_0_bins.size-1):
            r_inner = r_0_bins[i]
            r_outer = r_0_bins[i+1]
            aper = EllipticalAnnulus((self.isophote.x0, self.isophote.y0),
                                     r_inner, r_outer,
                                     r_outer * (1 - self.isophote.eps),
                                     theta=self.isophote.pa)
            mask = aper.to_mask(method='center').to_image(data.shape)
            try:
                lower_profile[i], median_profile[i], upper_profile[i] = np.nanpercentile(data[mask > 0], [16, 50, 84])
            except:
                lower_profile[i], median_profile[i], upper_profile[i] = (np.nan, np.nan, np.nan)

        if fig_name is not None:
            fig, axes = new_figure(f'{fig_name}_profile', figsize=(16, 4), ncols=4, sharey=False, sharex=False,
                                   gridspec_kw={'width_ratios': [1, 1, 1, 2]})
            ax = axes[0, 0]
            ax.set_title('data')
            im = ax.imshow(data, origin='lower', interpolation='nearest', cmap='nipy_spectral', norm=colors.LogNorm())
            cb = plt.colorbar(im, ax=ax, shrink=.5)
            
            ax = axes[0, 1]
            ax.set_title('model')
            model = np.interp(self.r_0, r_0_mid, median_profile)
            im = ax.imshow(model, origin='lower', interpolation='nearest', cmap='nipy_spectral', norm=im.norm)
            cb = plt.colorbar(im, ax=ax, shrink=.5)
            for i in range(r_0_bins.size-1):
                r_inner = r_0_bins[i]
                r_outer = r_0_bins[i+1]
                aper = EllipticalAnnulus((self.isophote.x0, self.isophote.y0),
                                         r_inner, r_outer,
                                         r_outer * (1 - self.isophote.eps),
                                         theta=self.isophote.pa)
                aper.plot(color='white', axes=ax)
            
            ax = axes[0, 2]
            ax.set_title('residual')
            residual = data-model
            mad = np.nanmedian(np.fabs(residual))
            im = ax.imshow(residual, origin='lower', interpolation='nearest', cmap='Spectral', norm=colors.Normalize(-5*mad, 5*mad))
            cb = plt.colorbar(im, ax=ax, shrink=.5)
            
            ax = axes[0, 3]
            ax.set_title('radial profile')
            ax.plot(self.r_0.ravel(), data.ravel(), 'c.', alpha=.05)
            ax.plot(r_0_mid, median_profile, 'r-+')
            ax.fill_between(r_0_mid, lower_profile, upper_profile, color='k', alpha=.5)
            #ax.set_ylim(-.1, .1)
            ax.set_yscale('log')
            fig.savefig(os.path.join(plots_dir, f"{fig_name}_profile.png"), facecolor='white')     
            if not show_plots:
                plt.close()
        
        return r_0_bins, median_profile, lower_profile, upper_profile